# Bronze Layer: Raw Data Ingestion
🎯 **Goal**: Collect and store the raw data exactly as it comes in

🔧 **Tasks a data engineer performs:**
- Write Python or Spark code to pull data from a source (API, file, database)
- Validate basic connectivity (check API responses, handle errors)
- Convert the raw response (e.g. JSON) to a DataFrame
- Store the data in a raw Delta table (or Parquet/CSV in simpler setups)
- Include basic metadata (e.g. ingestion timestamp)

### 📥 This notebook: Bronze Layer – Raw Pokémon Data Ingestion

In this notebook, we:

- Collect data from the [PokéAPI](https://pokeapi.co/) for the first 150 Pokémon
- Extract selected fields from the JSON response:
  - `id`, `name`, `height`, `weight`, `base_experience`, and `types`
- Convert the API responses into a Spark DataFrame
- Save the raw data to a Delta table in the **bronze layer** at `./data/bronze/pokemon`

🟫 No transformations are applied — we store the data as-is to preserve its original structure.
